# This file makes an API call to get the data for all starlink statellites

## 1. dependencies

In [11]:
import requests
import json
import configparser
import xlsxwriter
import time
from datetime import datetime
import pandas as pd

## 2. API building

In [12]:
uriBase                = "https://www.space-track.org"
requestLogin           = "/ajaxauth/login"
requestCmdAction       = "/basicspacedata/query" 
requestFindStarlinks   = "/class/tle_latest/NORAD_CAT_ID/>40000/ORDINAL/1/OBJECT_NAME/STARLINK~~/format/json/orderby/NORAD_CAT_ID%20asc"

## 3. Read Config file

In [ ]:
config = configparser.ConfigParser()
config.read("./SLTrack.ini")
configUsr = config.get("configuration","username")
configPwd = config.get("configuration","password")  
configOut = config.get("configuration","output")
siteCred = {'identity': configUsr, 'password': configPwd}

## 4. Making the API request and download the JSON file




In [19]:
class MyError(Exception):
    def __init___(self,args):
        Exception.__init__(self,"my exception was raised with arguments {0}".format(args))
        self.args = args



wsline = 1

with requests.Session() as session:
    # Log in
    resp = session.post(uriBase + requestLogin, data=siteCred)
    if resp.status_code != 200:
        raise MyError(resp, "POST fail on login")
    
    print("Login successful!")

    # Query for Starlink satellites
    print(uriBase + requestCmdAction + requestFindStarlinks) #print the satellite query for verification
    resp = session.get(uriBase + requestCmdAction + requestFindStarlinks)
    if resp.status_code != 200:
        print(resp)
        raise MyError(resp, "GET fail on request for Starlink satellites")

    print("Data retrieval successful!")

    # Parse response JSON
    retData = json.loads(resp.text)

    with open('Starlink_Satellites.json', 'w') as json_file: #save the json file
        json.dump(retData, json_file, indent=4)

    print("JSON data has been saved as 'Starlink_Satellites.json'")

Login successful!
https://www.space-track.org/basicspacedata/query/class/tle_latest/NORAD_CAT_ID/>40000/ORDINAL/1/OBJECT_NAME/STARLINK~~/format/json/orderby/NORAD_CAT_ID%20asc
Data retrieval successful!
JSON data has been saved as 'Starlink_Satellites.json'
